In [12]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from dotenv import load_dotenv
import os

load_dotenv()
api_key=os.getenv("GROQ_API_KEY")

model_client = OpenAIChatCompletionClient(
    api_key=api_key,
    base_url="https://api.groq.com/openai/v1",
    model="llama-3.3-70b-versatile",
    model_info={
        "family": "llama",
        "vision": False,
        "function_calling": True,
        "json_output": True,
    },
)

e:\Udemy Courses\Autogen_Crash_Course\.venv\Lib\site-packages\autogen_ext\models\openai\_openai_client.py:466: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


In [13]:
async def web_search(query: str) -> str:
    """Find information on the web. Currently using mock response"""
    return "The Labrador Retriever or simply Labrador is a British breed of retriever gun dog."

In [14]:
agent = AssistantAgent(
    name='Assistant',
    model_client=model_client,
    tools=[web_search],
    system_message="Use Tools to solve tasks",
    description="An agent that uses tools to solve tasks."
)

In [15]:
result = await agent.run(task="Find information about Labrador Retriever")
print(result.messages[-1].content)

The Labrador Retriever or simply Labrador is a British breed of retriever gun dog.


In [16]:
from autogen_core import CancellationToken

async def assistant_run()-> None:
    response = await agent.on_messages(
        messages=[TextMessage(content="Find information about Labrador Retriever", source='User')],
        cancellation_token=CancellationToken()
    )

    print(response.inner_messages)
    print('\n\n\n')
    print(response.chat_message)

await assistant_run()

[ToolCallRequestEvent(id='caf8e740-1852-457d-9ad9-bad321861c51', source='Assistant', models_usage=RequestUsage(prompt_tokens=304, completion_tokens=21), metadata={}, created_at=datetime.datetime(2025, 12, 16, 13, 24, 20, 592166, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='mkx9wwyce', arguments='{"query":"Labrador Retriever breed information"}', name='web_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='f240901b-2e64-456b-acac-fd44b1ad13f0', source='Assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 12, 16, 13, 24, 20, 595052, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='The Labrador Retriever or simply Labrador is a British breed of retriever gun dog.', name='web_search', call_id='mkx9wwyce', is_error=False)], type='ToolCallExecutionEvent')]




id='45991c67-aa7b-4d50-8c79-cf039908f74f' source='Assistant' models_usage=None metadata={} created_at=datetime.datetime(2025, 12, 16, 13, 24, 20, 595155, tz

In [19]:
from autogen_agentchat.ui import Console

async def assistant_run_stream() -> None:
    await Console(
        agent.on_messages_stream(
            messages=[TextMessage(content="Find information about the labrador retriever", source="User")],
            cancellation_token=CancellationToken()
        ),
        output_stats=True
    )

await assistant_run_stream()

---------- ToolCallRequestEvent (Assistant) ----------
[FunctionCall(id='dehc6px4v', arguments='{"query":"labrador retriever"}', name='web_search')]
[Prompt tokens: 381, Completion tokens: 18]
---------- ToolCallExecutionEvent (Assistant) ----------
[FunctionExecutionResult(content='The Labrador Retriever or simply Labrador is a British breed of retriever gun dog.', name='web_search', call_id='dehc6px4v', is_error=False)]
---------- Assistant ----------
The Labrador Retriever or simply Labrador is a British breed of retriever gun dog.
---------- Summary ----------
Number of inner messages: 2
Total prompt tokens: 381
Total completion tokens: 18
Duration: 0.33 seconds


In [20]:
from autogen_agentchat.ui import Console

async def assistant_run_stream() -> None:
    await Console(
        agent.on_messages_stream(
            messages=[TextMessage(content="What was the last question I asked you?", source="User")],
            cancellation_token=CancellationToken()
        ),
        output_stats=True
    )

await assistant_run_stream()

---------- Assistant ----------
Your last question was "Find information about the labrador retriever"
[Prompt tokens: 440, Completion tokens: 15]
---------- Summary ----------
Number of inner messages: 0
Total prompt tokens: 440
Total completion tokens: 15
Duration: 0.24 seconds
